Este colab fue desarrollado por Arnold Charry Armero.

# Regularización en Regresión Lineal Múltiple

La regularización es un conjunto de técnicas empleadas en el aprendizaje automático para evitar o reducir el sobreajuste de los modelos. Su objetivo principal es disminuir la varianza y la sensibilidad del modelo a los datos de entrenamiento, logrando un equilibrio entre sesgo y varianza.
En el caso de la Regresión Lineal Múltiple, la regularización incorpora un término de penalización en la función de costo para controlar la magnitud de los coeficientes y hacer que el modelo generalice mejor ante datos nuevos.
Las tres técnicas más comunes son Ridge (L2), que reduce la varianza y mitiga los efectos de la multicolinealidad; Lasso (L1), que además de reducir la varianza realiza una selección automática de variables; y Elastic Net, que combina las ventajas de ambas penalizaciones.
Se importan las librerias, la base de datos y la matriz de características para manejar todas las regularizaciones.

In [ ]:
# Se importan las librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Machine Learning/Bases de Datos/Student_Performance.csv')

In [ ]:
df.head()

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
0,7,99,Yes,9,1,91.0
1,4,82,No,4,2,65.0
2,8,51,Yes,7,2,45.0
3,5,52,Yes,5,2,36.0
4,7,75,No,8,5,66.0


Se analizan cuántos datos se tienen,

In [ ]:
# Número de datos
df.shape

(10000, 6)

In [ ]:
# Obtenemos las características
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

## Preprocesamiento de Datos

In [ ]:
# Se detectan las columnas categóricas
cat_cols = df.select_dtypes(include=['object', 'category']).columns
cat_indices = [df.columns.get_loc(col) for col in cat_cols]

# Se detectan las columnas numéricas
num_indices = [i for i in range(df.shape[1] - 1) if i not in cat_indices]

# Se crea el transformador
ct = ColumnTransformer(
    transformers=[('num', StandardScaler(), num_indices),
                  ('encoder', OneHotEncoder(drop='first',sparse_output=False, dtype=int), cat_indices)],
                    remainder='passthrough')

## Separación en Base de datos de Entrenamiento y Prueba

In [ ]:
# Se divide la base de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Escalado de Datos

In [ ]:
# Se escalan las variables
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [ ]:
# Visualizar el array X_train
print(X_train)

[[ 0.01177784  1.46795645 -0.31740964 -0.20579059  0.        ]
 [-1.14669864  1.63989277  0.2720233  -0.20579059  0.        ]
 [-1.14669864 -0.25140672  0.86145625 -1.25196174  1.        ]
 ...
 [-1.53285746  0.37902645 -0.31740964  0.14293313  1.        ]
 [-0.37438098 -1.45496093 -1.49627552  1.18910429  0.        ]
 [ 1.17025433  0.78021119  0.86145625 -1.25196174  1.        ]]


# Regresión Ridge (L2)

La Regresión Ridge introduce un término de penalización controlado por el parámetro
$λ$, que actúa sobre el cuadrado de los coeficientes $β_{j}^{2}$. Este término tiene como propósito reducir la magnitud de los coeficientes $β_{j}$ y, con ello, mitigar los efectos de la multicolinealidad entre las variables independientes.

La multicolinealidad afecta negativamente un modelo de Regresión Lineal de dos formas principales. En primer lugar, incrementa la varianza y la sensibilidad del modelo a los datos de entrenamiento, lo que dificulta su capacidad de generalizar adecuadamente ante nuevos datos. En segundo lugar, cuando dos o más variables explicativas están altamente correlacionadas, sus coeficientes “se pisan” entre sí, ya que el modelo no puede determinar con precisión cuál de ellas explica realmente la variación en la variable dependiente. Esto genera coeficientes inestables y de difícil interpretación, pues el modelo reparte arbitrariamente la influencia entre las variables correlacionadas, en lugar de reflejar su verdadera relación con la variable de respuesta.

De esta manera, Ridge regulariza el modelo reduciendo la varianza a costa de introducir un pequeño sesgo, pero obteniendo un mejor equilibrio entre ambos (bias-variance trade-off) y, por tanto, un modelo más estable y con mayor capacidad predictiva. Matemáticamente se define de la siguiente manera:

$$ \text{min}\: \sum_{i=1}^{n}\left ( y - \beta_{0} - \sum_{j=1}^{p} \beta_{j}x_{ij} \right )^{2} + \lambda \sum_{j=1}^{p} \beta_{j}^{2} $$

Ahora se continúa con la implementación en código,

In [ ]:
# Se codifican las categóricas
X_encoded = pd.get_dummies(df.iloc[:, :-1], drop_first=True)
X_encoded = X_encoded.astype(float)

# Calculamos el VIF
VIF = pd.DataFrame()
VIF["Variable"] = X_encoded.columns
VIF["VIF"] = [variance_inflation_factor(X_encoded.values, i) for i in range(X_encoded.shape[1])]
display(VIF)

,Variable,VIF
0,Hours Studied,4.263244
1,Previous Scores,10.088586
2,Sleep Hours,9.833594
3,Sample Question Papers Practiced,3.350481
4,Extracurricular Activities_Yes,1.928235


Si el VIF tiene un valor mayor a 10, eso quiere decir que hay una fuerte multicolinealidad y es importante usar Regresión Ridge.

## Selección del Lambda

In [ ]:
alphas = np.random.uniform(0, 10, 50)
ridge_cv = RidgeCV(alphas = alphas, cv = 10, scoring = 'neg_mean_squared_error')
ridge_cv.fit(X_train, y_train)

RidgeCV(alphas=array([8.6542512 , 8.93329931, 1.89618901, 9.59176042, 6.57398725,
       4.23578095, 6.31543599, 0.04563088, 1.01606732, 9.45143158,
       4.38043815, 9.4056439 , 4.68799795, 2.69505342, 1.5322561 ,
       8.46968602, 4.76671704, 1.90382823, 7.43877964, 2.33742645,
       3.25078066, 1.31877945, 9.14902541, 3.55967784, 8.18829962,
       5.53285641, 8.53835602, 8.11856709, 6.60483859, 8.16039102,
       9.71463061, 0.11476413, 1.31939948, 6.3105417 , 8.31355279,
       8.08234165, 6.32515624, 8.75995088, 8.43455955, 3.03370877,
       7.84543735, 1.36639646, 6.53299863, 9.63975919, 7.63256068,
       7.33213868, 2.29011669, 4.22222759, 5.80856098, 7.22391916]),
        cv=10, scoring='neg_mean_squared_error')

In [ ]:
# Mejor hiperparámetro alpha
alpha = float(ridge_cv.alpha_)
print(f"El mejor alpha es {alpha}")

El mejor alpha es 0.1147641271945199


Ahora se entrena el modelo,

## Entrenamiento del Modelo

In [ ]:
ridge_model = Ridge(alpha = alpha)
ridge_model.fit(X_train, y_train)

Ridge(alpha=0.1147641271945199)

Obteniendo los coeficientes del modelo,

In [ ]:
print('Coeficientes: ', ridge_model.coef_)
print('Intercepción: ', ridge_model.intercept_)

Coeficientes:  [ 7.38523338 17.77725712  0.80895596  0.53843477  0.66527265]
Intercepción:  54.77273919743349


In [ ]:
# Desescalado de coeficientes
scaler = ct.named_transformers_.get('num')
encoder = ct.named_transformers_.get('encoder')

# --- Identificar qué columnas son numéricas y cuáles categóricas ---
n_num = len(num_indices)
coef_scaled = ridge_model.coef_

# Separar coeficientes
coef_num_scaled = coef_scaled[:n_num]
coef_cat = coef_scaled[n_num:]

# --- Desescalar las numéricas ---
coef_num_original = coef_num_scaled / scaler.scale_
intercept_original = ridge_model.intercept_ - np.sum(scaler.mean_ * coef_num_scaled / scaler.scale_)

# --- Nombres de variables ---
num_names = df.columns[num_indices]
cat_feature_names = encoder.get_feature_names_out(cat_cols)

feature_names = list(num_names) + list(cat_feature_names)
coef_original = np.concatenate([coef_num_original, coef_cat])

# --- Tabla final ---
coef_table = pd.DataFrame({
    'Variable': feature_names,
    'Coeficiente (original)': coef_original
})

# Resultados
print('\n--- Coeficientes en escala original ---')
print(coef_table)
print('\nIntercepto en escala original:', intercept_original)



--- Coeficientes en escala original ---
                           Variable  Coeficiente (original)
0                     Hours Studied                2.851873
1                   Previous Scores                1.018852
2                       Sleep Hours                0.476825
3  Sample Question Papers Practiced                0.187765
4    Extracurricular Activities_Yes                0.665273

Intercepto en escala original: -34.07393510526304


Que, se debe tomar en cuenta, que el último coeficiente corresponde a la variable dummy o binaria por el orden en el que se hizo el transformador.

Realizando una predicción,

In [ ]:
# Se debe de hacer en el orden original
print("Predicción:", ridge_model.predict(ct.transform([[8, 90, 'Yes', 9, 3]]))[0])

Predicción: 85.95772794840441


In [ ]:
# Obtenemos las predicciones
y_pred = ridge_model.predict(X_test)
print(y_pred.reshape(len(y_pred),1))

[[50.45136619]
 [53.09382763]
 [78.24470294]
 ...
 [64.56849075]
 [25.89762196]
 [18.82650904]]


## Rendimiento del Modelo

In [ ]:
# KPI's del Modelo
MAE = mean_absolute_error(y_test, y_pred)
print('MAE: {:0.2f}%'.format(MAE / np.mean(y_test) * 100))
MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
print('RMSE: {:0.2f}%'.format(RMSE / np.mean(y_test) * 100))
r2 = r2_score(y_test, y_pred)
print('R2: {:0.2f}'.format(r2))

MAE: 2.91%
RMSE: 3.64%
R2: 0.99


## Validación Cruzada

In [ ]:
# Aplicar K-fold Cross Validation
scores = cross_val_score(estimator = ridge_model, X = X_train, y = y_train, cv = 10, scoring = 'neg_mean_squared_error')
print(np.sqrt(-scores.mean()))

2.0424951517358005


# Regresión Lasso (L1)

La Regresión Lasso incorpora un término de penalización controlado por el parámetro $λ$, que actúa sobre el valor absoluto de los coeficientes $|β_{j}|$. Este término tiene como objetivo reducir la magnitud de los coeficientes y, en muchos casos, llevar a cero aquellos que no aportan significativamente a la predicción.

El principal valor agregado de la Regresión Lasso es su capacidad para realizar una selección automática de variables, eliminando de forma implícita aquellas que no son relevantes dentro del conjunto de datos. Esto permite obtener modelos más simples y fácilmente interpretables, ya que se conservan únicamente las variables con mayor influencia sobre la variable dependiente.

Además, Lasso contribuye a reducir la varianza del modelo y a mejorar su capacidad de generalización, al introducir un pequeño sesgo controlado que equilibra la relación entre sesgo y varianza (bias-variance trade-off). De esta manera, se evita el sobreajuste a los datos de entrenamiento, logrando un modelo más robusto ante nuevos datos.

$$ \text{min}\: \sum_{i=1}^{n}\left ( y - \beta_{0} - \sum_{j=1}^{p} \beta_{j}x_{ij} \right )^{2} + \lambda \sum_{j=1}^{p} |\beta_{j}| $$

Ahora se continúa con la implementación en código,

In [ ]:
lasso_cv = LassoCV(eps = 0.001, n_alphas = 100, cv = 10)
lasso_cv.fit(X_train, y_train)

LassoCV(cv=10)

In [ ]:
# Mejor hiperparámetro alpha
alpha = float(lasso_cv.alpha_)
print(f"El mejor alpha es {alpha}")

El mejor alpha es 0.017756388573092054


Ahora se entrena el modelo,

## Entrenamiento del Modelo

In [ ]:
lasso_model = Lasso(alpha = alpha)
lasso_model.fit(X_train, y_train)

Lasso(alpha=0.017756388573092054)

Obteniendo los coeficientes del modelo,

In [ ]:
print('Coeficientes: ', lasso_model.coef_)
print('Intercepción: ', lasso_model.intercept_)

Coeficientes:  [ 7.36816389 17.76044615  0.7906332   0.52141557  0.59432002]
Intercepción:  54.80824212204316


In [ ]:
# Desescalado de coeficientes
scaler = ct.named_transformers_.get('num')
encoder = ct.named_transformers_.get('encoder')

# --- Identificar qué columnas son numéricas y cuáles categóricas ---
n_num = len(num_indices)
coef_scaled = lasso_model.coef_

# Separar coeficientes
coef_num_scaled = coef_scaled[:n_num]
coef_cat = coef_scaled[n_num:]

# --- Desescalar las numéricas ---
coef_num_original = coef_num_scaled / scaler.scale_
intercept_original = lasso_model.intercept_ - np.sum(scaler.mean_ * coef_num_scaled / scaler.scale_)

# --- Nombres de variables ---
num_names = df.columns[num_indices]
cat_feature_names = encoder.get_feature_names_out(cat_cols)

feature_names = list(num_names) + list(cat_feature_names)
coef_original = np.concatenate([coef_num_original, coef_cat])

# --- Tabla final ---
coef_table = pd.DataFrame({
    'Variable': feature_names,
    'Coeficiente (original)': coef_original
})

# Resultados
print('\n--- Coeficientes en escala original ---')
print(coef_table)
print('\nIntercepto en escala original:', intercept_original)


--- Coeficientes en escala original ---
                           Variable  Coeficiente (original)
0                     Hours Studied                2.845282
1                   Previous Scores                1.017889
2                       Sleep Hours                0.466025
3  Sample Question Papers Practiced                0.181830
4    Extracurricular Activities_Yes                0.594320

Intercepto en escala original: -33.84096501847777


In [ ]:
# Serie de coeficientes con nombres correctos
coef_lasso = pd.Series(lasso_model.coef_, index=feature_names)

# Variables eliminadas (coeficiente = 0)
vars_eliminadas = coef_lasso[coef_lasso == 0].index.tolist()

print("\nVariables eliminadas por Lasso:")
print(vars_eliminadas if vars_eliminadas else "Ninguna")


Variables eliminadas por Lasso:
Ninguna


Que, se debe tomar en cuenta, que el último coeficiente corresponde a la variable dummy o binaria por el orden en el que se hizo el transformador.

Realizando una predicción,

In [ ]:
# Se debe de hacer en el orden original
print("Predicción:", lasso_model.predict(ct.transform([[8, 90, 'Yes', 9, 3]]))[0])

Predicción: 85.86529527305967


In [ ]:
# Obtenemos las predicciones
y_pred = lasso_model.predict(X_test)
print(y_pred.reshape(len(y_pred),1))

[[50.47841215]
 [53.13151196]
 [78.26180113]
 ...
 [64.51732125]
 [25.94672129]
 [18.94026697]]


## Rendimiento del Modelo

In [ ]:
# KPI's del Modelo
MAE = mean_absolute_error(y_test, y_pred)
print('MAE: {:0.2f}%'.format(MAE / np.mean(y_test) * 100))
MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
print('RMSE: {:0.2f}%'.format(RMSE / np.mean(y_test) * 100))
r2 = r2_score(y_test, y_pred)
print('R2: {:0.2f}'.format(r2))

MAE: 2.91%
RMSE: 3.64%
R2: 0.99


## Validación Cruzada

In [ ]:
# Aplicar K-fold Cross Validation
scores = cross_val_score(estimator = lasso_model, X = X_train, y = y_train, cv = 10, scoring = 'neg_mean_squared_error')
print(np.sqrt(-scores.mean()))

2.043113768889395


# Regresión Elastic Net

La Regresión Elastic Net combina las penalizaciones de Ridge (L2) y Lasso (L1), buscando aprovechar las ventajas de ambas técnicas. Este enfoque reduce la varianza y mitiga los efectos de la multicolinealidad, al tiempo que puede llevar algunos coeficientes a cero para realizar una selección automática de variables. Elastic Net resulta especialmente útil cuando existen variables altamente correlacionadas o cuando el número de predictores es mayor que el número de observaciones. Matemáticamente se modela de la siguiente manera,

$$ \text{min}\: \sum_{i=1}^{n}\left ( y - \beta_{0} - \sum_{j=1}^{p} \beta_{j}x_{ij} \right )^{2} + \lambda \left [\alpha  \sum_{j=1}^{p} |\beta_{j}| + (1-\alpha ) \sum_{j=1}^{p} \beta_{j}^{2} \right ]  $$

Ahora se continúa con la implementación en código,

In [ ]:
elastic_net_cv = ElasticNetCV(eps = 0.001, n_alphas = 100, l1_ratio = [0.1, 0.5, 0.7, 0.8, 0.9, 0.95, 1],
                              cv = 10, max_iter = 100000)
elastic_net_cv.fit(X_train, y_train)

ElasticNetCV(cv=10, l1_ratio=[0.1, 0.5, 0.7, 0.8, 0.9, 0.95, 1],
             max_iter=100000)

In [ ]:
ratio = float(elastic_net_cv.l1_ratio_)
print(ratio)

1.0


In [ ]:
alpha = float(elastic_net_cv.alpha_)
print(alpha)

0.017756388573092054


Ahora se entrena el modelo,

## Entrenamiento del Modelo

In [ ]:
elastic_model = ElasticNet(alpha = alpha, l1_ratio = ratio)
elastic_model.fit(X_train, y_train)

ElasticNet(alpha=0.017756388573092054, l1_ratio=1.0)

Obteniendo los coeficientes del modelo,

In [ ]:
print('Coeficientes: ', elastic_model.coef_)
print('Intercepción: ', elastic_model.intercept_)

Coeficientes:  [ 7.36816389 17.76044615  0.7906332   0.52141557  0.59432002]
Intercepción:  54.80824212204316


In [ ]:
# Desescalado de coeficientes
scaler = ct.named_transformers_.get('num')
encoder = ct.named_transformers_.get('encoder')

# --- Identificar qué columnas son numéricas y cuáles categóricas ---
n_num = len(num_indices)
coef_scaled = elastic_model.coef_

# Separar coeficientes
coef_num_scaled = coef_scaled[:n_num]
coef_cat = coef_scaled[n_num:]

# --- Desescalar las numéricas ---
coef_num_original = coef_num_scaled / scaler.scale_
intercept_original = elastic_model.intercept_ - np.sum(scaler.mean_ * coef_num_scaled / scaler.scale_)

# --- Nombres de variables ---
num_names = df.columns[num_indices]
cat_feature_names = encoder.get_feature_names_out(cat_cols)

feature_names = list(num_names) + list(cat_feature_names)
coef_original = np.concatenate([coef_num_original, coef_cat])

# --- Tabla final ---
coef_table = pd.DataFrame({
    'Variable': feature_names,
    'Coeficiente (original)': coef_original
})

# Resultados
print('\n--- Coeficientes en escala original ---')
print(coef_table)
print('\nIntercepto en escala original:', intercept_original)


--- Coeficientes en escala original ---
                           Variable  Coeficiente (original)
0                     Hours Studied                2.845282
1                   Previous Scores                1.017889
2                       Sleep Hours                0.466025
3  Sample Question Papers Practiced                0.181830
4    Extracurricular Activities_Yes                0.594320

Intercepto en escala original: -33.84096501847777


Que, se debe tomar en cuenta, que el último coeficiente corresponde a la variable dummy o binaria por el orden en el que se hizo el transformador.

Realizando una predicción,

In [ ]:
# Se debe de hacer en el orden original
print("Predicción:", elastic_model.predict(ct.transform([[8, 90, 'Yes', 9, 3]]))[0])

Predicción: 85.86529527305967


In [ ]:
# Obtenemos las predicciones
y_pred = elastic_model.predict(X_test)
print(y_pred.reshape(len(y_pred),1))

[[50.47841215]
 [53.13151196]
 [78.26180113]
 ...
 [64.51732125]
 [25.94672129]
 [18.94026697]]


## Rendimiento del Modelo

In [ ]:
# KPI's del Modelo
MAE = mean_absolute_error(y_test, y_pred)
print('MAE: {:0.2f}%'.format(MAE / np.mean(y_test) * 100))
MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
print('RMSE: {:0.2f}%'.format(RMSE / np.mean(y_test) * 100))
r2 = r2_score(y_test, y_pred)
print('R2: {:0.2f}'.format(r2))

MAE: 2.91%
RMSE: 3.64%
R2: 0.99


## Validación Cruzada

In [ ]:
# Aplicar K-fold Cross Validation
scores = cross_val_score(estimator = elastic_model, X = X_train, y = y_train, cv = 10, scoring = 'neg_mean_squared_error')
print(np.sqrt(-scores.mean()))

2.043113768889395


## Referencias

*   Jacinto, V. R. (2024). Machine learning: Fundamentos, algoritmos y aplicaciones para los negocios, industria y finanzas. Ediciones Díaz de Santos.
*   James, G., Witten, D., Hastie, T., & Tibshirani, R. (2021). An Introduction to Statistical Learning: with Applications in R. https://link.springer.com/content/pdf/10.1007/978-1-0716-1418-1.pdf
*   Student performance (Multiple Linear regression). (2023, June 29). Kaggle. https://www.kaggle.com/datasets/nikhil7280/student-performance-multiple-linear-regression